# using sqlparse to create meta data

### instructions
- git pull latest

### limitations
- when using spaces in tables names with reserved keyworkds, e.g. case in "tableau enc case view"

Have solved this by changing sq_metadata

### how does this work?
- sql files is made up of one to many sql statements
- each sql statments is made up of tokens
- tokens seperate out keyworkds, DML, DDL, names and puncutation ect.

### output
- parent child relationships of every single sql statement in every sql file (can be multiple)
- example:

In [555]:
# example output 
[{'fileName': 'Tableau MTM Glide Path Target View.sql',
  'name': 'Tableau MTM Glide Path Target View',
  'path': '/home/davidgriffiths/csaa-aws-data-capability/nbn_dde_23/tableau_views/Tableau MTM Glide Path Target View.sql',
  'depnd_tables': [],
  'parent': 'nbn_dde_23."Tableau MTM Glide Path Target View"',
  'ddl_type': 'DROP'}]

[{'fileName': 'Tableau MTM Glide Path Target View.sql',
  'name': 'Tableau MTM Glide Path Target View',
  'path': '/home/davidgriffiths/csaa-aws-data-capability/nbn_dde_23/tableau_views/Tableau MTM Glide Path Target View.sql',
  'depnd_tables': [],
  'parent': 'nbn_dde_23."Tableau MTM Glide Path Target View"',
  'ddl_type': 'DROP'}]

# setup

In [556]:
!conda info --envs

# conda environments:
#
base                     /home/davidgriffiths/miniconda3
sqlparse              *  /home/davidgriffiths/miniconda3/envs/sqlparse



In [557]:
import sql_metadata

In [558]:
import re
from typing import List, Tuple, Optional, Dict
import sqlparse

from sqlparse.sql import TokenList
from sqlparse.tokens import Name, Whitespace, Wildcard, Number, Punctuation, DDL 

# re built get_query_tokens
need to account for multiple queries in a file

In [559]:
def get_query_tokens(query: str) -> List[sqlparse.sql.Token]:
    """
    daves changes: sqlparse.parse(query) returns a tuple if multiple queries ";" need to handle this
    :type query str
    :rtype: list[{queryNoWithinFile str,  tokens: list[sqlparse.sql.Token]}
    """
    query = sql_metadata.preprocess_query(query)
    parsed = sqlparse.parse(query)

    # handle empty queries (#12)
    if not parsed:
        return []

    queries_list_and_tokens =[]
    n=0
    for qry in parsed:
        tokens = TokenList(qry.tokens).flatten()
        # print([(token.value, token.ttype) for token in tokens])

        dic = {"queryNoWithinFile":n,"tokens":[token for token in tokens if token.ttype is not Whitespace]}
        queries_list_and_tokens.append(dic)
        n = n+1
    return queries_list_and_tokens

# re built get_query_tables
i remvoed the quote removal

In [560]:
def get_query_tables(query: str, dev=False) -> List[str]:
    """
    :type query str
    :rtype: list[str]
    """
    

    table_syntax_keywords = [
        # SELECT queries
        'FROM', 'WHERE', 'JOIN', 'INNER JOIN', 'FULL JOIN', 'FULL OUTER JOIN',
        'LEFT OUTER JOIN', 'RIGHT OUTER JOIN',
        'LEFT JOIN', 'RIGHT JOIN', 'ON',
        # INSERT queries
        'INTO', 'VALUES',
        # UPDATE queries
        'UPDATE', 'SET',
        # Hive queries
        'TABLE',  # INSERT TABLE
        
    ]
    
#     print(sql_metadata.get_query_tokens(query))
#     query = query.replace('"', '')
    query = sqlparse.format(query, strip_comments=True).strip()
#     print(query)
    queries_list_and_tokens = get_query_tokens(query)
    
    queries_list_and_tokens_parsed =[]
    
    for query_and_tokens in queries_list_and_tokens:
        parent_table_list = []
        parent_table_counter = 0 # increments to 2 to get schema and table
        tables = []
        last_keyword = None
        last_ddl =None
        keep_last_ddl = None
        
        for index, token in enumerate(query_and_tokens['tokens']):
            if dev==True: print([token, token.ttype, last_keyword,last_ddl, token.value.upper(),parent_table_counter, last_ddl])
            if token.is_keyword and token.value.upper() in table_syntax_keywords:
                # keep the name of the last keyword, the next one can be a table name
                last_keyword = token.value.upper()
#                 print('keyword', last_keyword)
            elif str(token.ttype) == 'Token.Keyword.DDL':
                # keep the name of the last create, the next one can be annother table to create
                last_ddl = token.value.upper()
                keep_last_ddl= token.value.upper()
#                 print('keyword.lld', token.value.upper())
            elif str(token) == '(':
                # reset the last_keyword for INSERT `foo` VALUES(id, bar) ...
                last_keyword = None
            elif token.is_keyword and str(token) in ['FORCE', 'ORDER', 'GROUP BY']:
                # reset the last_keyword for queries like:
                # "SELECT x FORCE INDEX"
                # "SELECT x ORDER BY"
                # "SELECT x FROM y GROUP BY x"
                last_keyword = None
            elif token.is_keyword and str(token) == 'SELECT' and last_keyword in ['INTO', 'TABLE']:
                # reset the last_keyword for "INSERT INTO SELECT" and "INSERT TABLE SELECT" queries
                last_keyword = None
            elif (token.ttype is Name or str(token.ttype) == 'Token.Literal.String.Symbol') and last_ddl in ['CREATE','DROP']:
#                 print("found a CREATE or DROP")
                parent_table_list.append(str(token))
                parent_table_counter = parent_table_counter + 1
                if parent_table_counter > 1:
                    last_ddl = None
                    parent_table_counter = 0
#                 parent_table = sql_metadata._update_table_names(tables, query_and_tokens['tokens'], index, last_keyword)
            elif token.ttype is Name or token.ttype or token.is_keyword:
                tables = sql_metadata._update_table_names(tables, query_and_tokens['tokens'], index, last_keyword)
        unique_tables = sql_metadata.unique(tables)
        dic = {
            "queryNoWithinFile": query_and_tokens['queryNoWithinFile'],
            "tables": unique_tables,
            "parent_table": '.'.join(parent_table_list),
            "ddl_type": keep_last_ddl
        }
        queries_list_and_tokens_parsed.append(dic)

    return queries_list_and_tokens_parsed

# working with multiple files

In [561]:
import os
from os import path
from pathlib import Path

# get all sql files

In [562]:
files = os.listdir()

In [563]:
files_filtered = list(filter(lambda x: re.search(r"^.*\.(sql)$", x), files))

# get all sql view depedncies

### get all files in RAPID

In [564]:
rapid_dirs = ['nbn_dde_23/tableau_views'] # ['tableau_dde/transform/views'] # ['nbn_dde_23/tableau_views','tableau_dde/transform/views']

In [565]:
rapid_files = []
for rapid_dir in rapid_dirs:
    for dirpath, dirname, filenames in os.walk(os.path.join('/home/davidgriffiths/csaa-aws-data-capability',rapid_dir)):
    #     print('Current path:', dirpath)
    #     print('Directories:', dirname)
    #     print('Files:', filenames)
        for file in filenames:
            file_dict = {
                "fileName": file,
                "path": os.path.join(dirpath,file),
            }
            rapid_files.append(file_dict)
rapid_files[0:3]

[{'fileName': 'Tableau MTM Glide Path Target View.sql',
  'path': '/home/davidgriffiths/csaa-aws-data-capability/nbn_dde_23/tableau_views/Tableau MTM Glide Path Target View.sql'},
 {'fileName': 'Tableau Incident Mdf Detailed View.sql',
  'path': '/home/davidgriffiths/csaa-aws-data-capability/nbn_dde_23/tableau_views/Tableau Incident Mdf Detailed View.sql'},
 {'fileName': 'Tableau Incident Work Requests View.sql',
  'path': '/home/davidgriffiths/csaa-aws-data-capability/nbn_dde_23/tableau_views/Tableau Incident Work Requests View.sql'}]

### keep only .sql files

In [566]:
rapid_files_filtered = list(filter(lambda x: re.search(r"^.*\.(sql)$", x['fileName']), rapid_files))
rapid_files_filtered[0:5]

[{'fileName': 'Tableau MTM Glide Path Target View.sql',
  'path': '/home/davidgriffiths/csaa-aws-data-capability/nbn_dde_23/tableau_views/Tableau MTM Glide Path Target View.sql'},
 {'fileName': 'Tableau Incident Mdf Detailed View.sql',
  'path': '/home/davidgriffiths/csaa-aws-data-capability/nbn_dde_23/tableau_views/Tableau Incident Mdf Detailed View.sql'},
 {'fileName': 'Tableau Incident Work Requests View.sql',
  'path': '/home/davidgriffiths/csaa-aws-data-capability/nbn_dde_23/tableau_views/Tableau Incident Work Requests View.sql'},
 {'fileName': 'Tableau Genesys VWFM Agent Adherence View.sql',
  'path': '/home/davidgriffiths/csaa-aws-data-capability/nbn_dde_23/tableau_views/Tableau Genesys VWFM Agent Adherence View.sql'},
 {'fileName': 'Tableau Pni Snapshot Audit Report View.sql',
  'path': '/home/davidgriffiths/csaa-aws-data-capability/nbn_dde_23/tableau_views/Tableau Pni Snapshot Audit Report View.sql'}]

## loop through files

In [567]:
list_of_rapid_sql_views = []

for file in rapid_files_filtered:
    with open(file['path'], 'r') as f:
        query = f.read()
        queries_and_tables = get_query_tables(query,False)
    
        for qry in queries_and_tables:
            
            dic = {
                'fileName': file['fileName'],
                'name': os.path.splitext(file['fileName'])[0],
                'path': file['path'].replace("/home/davidgriffiths", ""),
                'depnd_tables': qry['tables'],
                'parent': qry["parent_table"],
                'ddl_type': qry['ddl_type']
            }

            list_of_rapid_sql_views.append(dic) 
        

list_of_rapid_sql_views[0:5]

[{'fileName': 'Tableau MTM Glide Path Target View.sql',
  'name': 'Tableau MTM Glide Path Target View',
  'path': '/csaa-aws-data-capability/nbn_dde_23/tableau_views/Tableau MTM Glide Path Target View.sql',
  'depnd_tables': [],
  'parent': 'nbn_dde_23."Tableau MTM Glide Path Target View"',
  'ddl_type': 'DROP'},
 {'fileName': 'Tableau MTM Glide Path Target View.sql',
  'name': 'Tableau MTM Glide Path Target View',
  'path': '/csaa-aws-data-capability/nbn_dde_23/tableau_views/Tableau MTM Glide Path Target View.sql',
  'depnd_tables': ['nbn_dde_23.csa_mtm_glide_path_target_extract_t'],
  'parent': 'nbn_dde_23."Tableau MTM Glide Path Target View"',
  'ddl_type': 'CREATE'},
 {'fileName': 'Tableau Incident Mdf Detailed View.sql',
  'name': 'Tableau Incident Mdf Detailed View',
  'path': '/csaa-aws-data-capability/nbn_dde_23/tableau_views/Tableau Incident Mdf Detailed View.sql',
  'depnd_tables': [],
  'parent': 'nbn_dde_23."Tableau Incident Mdf Detailed View"',
  'ddl_type': 'DROP'},
 {'fi

# write out as JSON

In [568]:
import json

In [569]:
# # loads json from string
# datajsonstr=json.loads('{"str":"str"}')
# datajsonstr

In [570]:
# stringify = json.dumps(list_of_rapid_sql_views, indent=2)
# stringify

In [571]:
# with open('jsonfile.json') as f:
#     data = json.load(f)

In [572]:
with open('list_of_rapid_sql_views.json','w') as f:
    json.dump(list_of_rapid_sql_views, f)

# test out funny files

In [573]:
filenamedave = '/home/davidgriffiths/csaa-aws-data-capability/nbn_dde_23/tableau_views/Tableau Incident WO Links View.sql'

In [574]:
with open(filenamedave, 'r') as f:
        query = f.read()
#         print(query)
        tables = get_query_tables(query)
        print(tables)

[{'queryNoWithinFile': 0, 'tables': [], 'parent_table': 'nbn_dde_23."Tableau Incident WO Links View"', 'ddl_type': 'DROP'}, {'queryNoWithinFile': 1, 'tables': ['nbn_dde_23.CSA_INCIDENT_LINKS_WO_T'], 'parent_table': '', 'ddl_type': 'CREATE  OR REPLACE'}]


# problem sqlparse.parse doesnt do multiple queries

In [575]:
with open(filenamedave, 'r') as f:
        query = f.read()
#         print(query)
#         tables = get_query_tables(query)
#         print(tables)

        # is parsing within sql_metadata.get_query_tokens working
#         res = sqlparse.parse(query)
#         print(res[1]) ##  this second query isnt getting picked up
        
        print(get_query_tokens(query))
        
        

[{'queryNoWithinFile': 0, 'tokens': [<DDL 'drop' at 0x7F7FEC057288>, <Keyword 'view' at 0x7F7FEC057408>, <Keyword 'if' at 0x7F7FEC0574C8>, <Keyword 'exists' at 0x7F7FEC057888>, <Name 'nbn_dd...' at 0x7F7FEC057948>, <Punctuation '.' at 0x7F7FEC0579A8>, <Symbol '"Table...' at 0x7F7FEC057CA8>, <Punctuation ';' at 0x7F7FEC057D08>]}, {'queryNoWithinFile': 1, 'tokens': [<DDL 'CREATE...' at 0x7F7FDDBC4B88>, <Keyword 'VIEW' at 0x7F7FDDBC4EE8>, <Name 'nbn_dd...' at 0x7F7FDDBC4048>, <Punctuation '.' at 0x7F7FDDBC40A8>, <Symbol '"Table...' at 0x7F7FDD4E61C8>, <Keyword 'AS' at 0x7F7FDD4E6A68>, <DML 'SELECT' at 0x7F7FDD4E6288>, <Name 'INC_WO...' at 0x7F7FDD4E6D68>, <Punctuation ',' at 0x7F7FDD4E60A8>, <Name 'INCIDE...' at 0x7F7FDD4E6F48>, <Punctuation ',' at 0x7F7FDD4E6708>, <Name 'APPOIN...' at 0x7F7FDD4E6E28>, <Punctuation ',' at 0x7F7FDD4E6408>, <Name 'WORK_R...' at 0x7F7FEC057E88>, <Punctuation ',' at 0x7F7FEC0577C8>, <Name 'WORK_O...' at 0x7F7FEC0576A8>, <Punctuation ',' at 0x7F7FEC057228>, <N